In [45]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate

# ChatOpenAI initialize
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=False,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

# Few-shot examples for movie to emoji conversion (Marvel movies!!)
examples = [
    {"input": "Spider-Man", "output": "🕷️🕸️👨"},
    {"input": "Iron Man", "output": "🤖⚡🔴"},
    {"input": "Thor", "output": "⚡🔨👨"},
    {"input": "Captain America", "output": "🛡️⭐🇺🇸"},
    {"input": "Avengers", "output": "🦸‍♂️🌍💥"},
]

# Create example prompt
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

# Create few-shot prompt template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Create the main prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI that converts movie titles into exactly 3 emojis."),
    MessagesPlaceholder(variable_name="history"),
    few_shot_prompt,
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(f"content='{result.content}'")
    return result

In [46]:
invoke_chain("Black Panther")


content='🐾👑⚔️'


AIMessage(content='🐾👑⚔️')

In [47]:
invoke_chain("Doctor Strange")


content='🔮🧙‍♂️⏳'


AIMessage(content='🔮🧙\u200d♂️⏳')

In [48]:
invoke_chain("What was the first movie I asked about?")

content='🐾👑⚔️'


AIMessage(content='🐾👑⚔️')

In [49]:
print(memory.load_memory_variables({}))


{'history': [HumanMessage(content='Black Panther'), AIMessage(content='🐾👑⚔️'), HumanMessage(content='Doctor Strange'), AIMessage(content='🔮🧙\u200d♂️⏳'), HumanMessage(content='What was the first movie I asked about?'), AIMessage(content='🐾👑⚔️')]}


In [50]:
invoke_chain("Titanic")


content='🚢❄️💔'


AIMessage(content='🚢❄️💔')